<a href="https://colab.research.google.com/github/srRyzhov/Hackathon/blob/main/NLP_DeepPavlov7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -U accelerate
! pip install -U transformers

In [ ]:
import pandas as pd
import numpy as np
import random
import torch

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from transformers import BertForSequenceClassification, BertTokenizer
from transformers import TrainingArguments
from transformers import Trainer

import transformers
import accelerate
transformers.__version__, accelerate.__version__


('4.33.1', '0.22.0')

Загрузить файлы: CRA_train_961.xlsx и CRA_test_239.xlsx

In [ ]:
train_df = pd.read_excel('/content/CRA_train_961.xlsx', engine = 'openpyxl', index_col = 0)
train_text = train_df['pr_txt']
train_labels = train_df['Категория']
#train_labels = train_df['Уровень рейтинга']

train_df = pd.read_excel('/content/CRA_test_239.xlsx', engine = 'openpyxl', index_col = 0)
test_text = train_df['pr_txt']
test_labels = train_df['Категория']
#test_labels = train_df['Уровень рейтинга']


Тексты для классификации - train_text

Метки классов, соответствующие текстам - train_labels

In [ ]:
def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
seed_all(42)

In [ ]:
# Модель 'DeepPavlov/rubert-base-cased'  Внимание: 7 или 17 выбрать
model_name = 'DeepPavlov/rubert-base-cased'
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=7) # 7!!!!
tokenizer = BertTokenizer.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokens_train = tokenizer.batch_encode_plus(
    train_text.values,
    max_length = 512,
    padding = 'max_length',
    truncation = True
)

tokens_test = tokenizer.batch_encode_plus(
    test_text.values,
    max_length = 512,
    padding = 'max_length',
    truncation = True
)

In [ ]:
# Создание и обучение кодировщика на тренировочных метках
label_encoder = LabelEncoder()
label_encoder.fit(train_labels)

# Преобразование тренировочных и тестовых меток в целочисленные значения
train_labels_encoded = label_encoder.transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Получение соответствия между исходными метками и их целочисленными значениями
label_mapping = {label: value for label, value in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))}


In [ ]:
#оборачиваем токенизированные текстовые данные в torch Dataset:
class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item
    def __len__(self):
        return len(self.labels)

#train_dataset = Data(tokens_train, train_labels)
#test_dataset = Data(tokens_test, test_labels)

train_dataset = Data(tokens_train, train_labels_encoded)
test_dataset = Data(tokens_test, test_labels_encoded)

In [ ]:
#расчет метрики - F1

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='micro')
    # one of [None, 'micro', 'macro', 'weighted']
    return {'F1': f1}

In [ ]:
#параметры для обучения:
training_args = TrainingArguments(
    output_dir = './results', #Выходной каталог
    num_train_epochs = 20, #Кол-во эпох для обучения
    per_device_train_batch_size = 14, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 14, #Размер пакета для каждого устройства во время валидации
    weight_decay =0.01, #Понижение весов
    logging_dir = './logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = 1e-5, #Скорость обучения
    evaluation_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 1,
    seed=42)

In [ ]:
#Передача в trainer предообученной модели, tokenizer, данных для обучения, данных для валидации и способа расчета метрики
trainer = Trainer(model=model,
                  tokenizer = tokenizer,
                  args = training_args,
                  train_dataset = train_dataset,
                  eval_dataset = test_dataset,
                  compute_metrics = compute_metrics)

In [ ]:
#Запуск обучения модели
trainer.train()

In [ ]:
#Сохранение обученной модели
model_path = "fine-tune-bert"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

In [ ]:
#функция для получения предикта
def get_prediction():
    test_pred = trainer.predict(test_dataset)
    labels = np.argmax(test_pred.predictions, axis = -1)
    return labels
pred = get_prediction()

In [ ]:
#проверка полученного результата

In [ ]:
#оценки качества модели
print(classification_report(test_labels_encoded, pred))
print(f1_score(test_labels_encoded, pred, average='micro'))

              precision    recall  f1-score   support

           0       0.61      0.81      0.70        68
           1       0.78      0.57      0.66        37
           2       0.90      0.93      0.92        41
           3       0.00      0.00      0.00        12
           4       0.27      0.12      0.16        26
           5       0.54      0.73      0.62        52
           6       0.00      0.00      0.00         4

    accuracy                           0.65       240
   macro avg       0.44      0.45      0.44       240
weighted avg       0.59      0.65      0.61       240

0.6458333333333334


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
